 Детальная и комплексная кластеризация спиртных напитков объемом 0.7 л, которые продаются в сети ["КуулКлевер"](https://www.coolclever.ru/catalog/otdokhni/all?filters=11099%255B0%255D%3D10%2611099%255B1%255D%3D11%2611099%255B2%255D%3D9%2611099%255B3%255D%3D12%2611099%255B4%255D%3D17%2611099%255B5%255D%3D13%2611099%255B6%255D%3D14%2611099%255B7%255D%3D15%2611099%255B8%255D%3D16%2611099%255B9%255D%3D43%2611109%255B0%255D%3D0.700&sort=cheap&page=9), с помощью различных методов машинного обучения.


In [23]:
# Объявляю нужные библиотеки
import pandas as pd # Библиотека для работы с фреймворками

In [24]:
# Загрузка датасета из csv-файла
df = pd.read_csv('CoolClever.csv', sep=',')
df

,Напиток,Цена,Страна,Крепость,Выдержка,Подарочная упаковка
0,Водка,490,Россия,40.0,0,0
1,Водка,490,Россия,40.0,0,0
2,Водка,490,Россия,40.0,0,0
3,Водка,539,Россия,40.0,0,0
4,Водка,539,Россия,40.0,0,0
5,Водка,549,Россия,40.0,0,0
6,Водка,599,Россия,40.0,0,0
7,Водка,649,Россия,40.0,0,0
8,Джин,659,Россия,40.0,0,0
9,Джин,659,Россия,40.0,0,0
